In [2]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [3]:
all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1795 rows x 4 columns]


## 三種買賣策略

In [4]:
def strategyM11_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    print(close_price.iloc[0])
    print(close_price.iloc[1])
    if close_price.iloc[0] - close_price.iloc[1] > 1.5:
        return 0
    else:
        initial_price = close_price.iloc[1]
    allprofit = 0
    remain_position = 1
    condition12_remain = True
    condition5MA_remain = True
    condition10MA_remain = True
    for i in range(len(close_price)-2):
        profit = (close_price.iloc[i+2]-initial_price)/initial_price
        conditionstoploss = profit <= -0.05
        condition12 = profit > 0.12
        condition5MA = close_price.iloc[i+2] < stock.loc[stock.index>day, '5MA'].iloc[i+2]
        condition10MA = close_price.iloc[i+2] < stock.loc[stock.index>day, '10MA'].iloc[i+2]
        if conditionstoploss:
            print('stoploss')
            return profit
        if any([condition12, condition5MA, condition10MA]):
            if condition12 and condition12_remain:
                allprofit += profit*0.3
                condition12_remain = False
                remain_position -= 0.3
                print('condition12', profit*0.3)
            if condition5MA and condition5MA_remain:
                allprofit += profit*0.3
                condition5MA_remain = False
                remain_position -= 0.3
                print('condition5MA', profit*0.3)
            if condition10MA and condition10MA_remain:
                allprofit += profit*0.4
                condition10MA_remain = False
                remain_position -= 0.4
                print('condition10MA', profit*0.4)
        if remain_position == 0:
            return allprofit
    allprofit += profit*remain_position
    return allprofit
def strategyM21_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    if close_price.iloc[0] - close_price.iloc[1] > 1.5:
        return 0
    else:
        initial_price = close_price.iloc[1]
    allprofit = 0
    remain_position = 1
    condition15_remain = True
    condition10MA_remain = True
    condition20MA_remain = True
    for i in range(len(close_price)-2):
        profit = (close_price.iloc[i+2]-initial_price)/initial_price
        conditionstoploss = profit <= -0.06
        condition15 = profit > 0.15
        condition10MA = close_price.iloc[i+2] < stock.loc[stock.index>day, '10MA'].iloc[i+2]
        condition20MA = close_price.iloc[i+2] < stock.loc[stock.index>day, '20MA'].iloc[i+2]
        if conditionstoploss:
            print('stoploss')
            return profit
        if any([condition15, condition10MA, condition20MA]):
            if condition15 and condition15_remain:
                allprofit += profit*0.3
                condition15_remain = False
                remain_position -= 0.3
                print('condition15', profit*0.3)
            if condition10MA and condition10MA_remain:
                allprofit += profit*0.3
                condition10MA_remain = False
                remain_position -= 0.3
                print('condition10MA', profit*0.3)
            if condition20MA and condition20MA_remain:
                allprofit += profit*0.4
                condition20MA_remain = False
                remain_position -= 0.4
                print('condition20MA', profit*0.4)
        if remain_position == 0:
            return allprofit
    allprofit += profit*remain_position
    return allprofit
def strategyM31_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    if close_price.iloc[0] - close_price.iloc[1] > 1.5:
        return 0
    else:
        initial_price = close_price.iloc[1]
    allprofit = 0
    remain_position = 1
    condition15_remain = True
    condition10MA_remain = True
    condition20MA_remain = True
    for i in range(len(close_price)-2):
        profit = (close_price.iloc[i+2]-initial_price)/initial_price
        conditionstoploss = profit <= -0.06
        condition15 = profit > 0.15
        condition10MA = close_price.iloc[i+2] < stock.loc[stock.index>day, '10MA'].iloc[i+2]
        condition20MA = close_price.iloc[i+2] < stock.loc[stock.index>day, '20MA'].iloc[i+2]
        if conditionstoploss:
            print('stoploss')
            return profit
        if any([condition15, condition10MA, condition20MA]):
            if condition15 and condition15_remain:
                allprofit += profit*0.3
                condition15_remain = False
                remain_position -= 0.3
                print('condition15', profit*0.3)
            if condition10MA and condition10MA_remain:
                allprofit += profit*0.2
                condition10MA_remain = False
                remain_position -= 0.2
                print('condition10MA', profit*0.2)
            if condition20MA and condition20MA_remain:
                allprofit += profit*0.5
                condition20MA_remain = False
                remain_position -= 0.5
                print('condition20MA', profit*0.5)
        if remain_position == 0:
            return allprofit
    allprofit += profit*remain_position
    return allprofit
def strategy9_profit(stock:pd.DataFrame, day, day1M):
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    MA10 = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), '10MA']
    initial_price = close_price.iloc[1]
    profit1 = 0
    for i in range(len(close_price)-2):
        profit = (close_price.iloc[i+2]-initial_price)/initial_price
        if profit1 != 0:
            if close_price.iloc[i+2] < MA10.iloc[i+2]:
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.06:
                profit = -0.06
                break
            elif profit > 0.15:
                profit1 = 0.15
    return profit
# def strategy5_profit(stock, day, day1M):
#     initial_price = stock.loc[stock.index>day, 'Adj Close']
#     close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
#     low_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Low']
#     initial_price = close_price.iloc[0]
#     profit1 = 0
#     last_low = low_price.iloc[0]
#     for i in range(len(close_price)-1):
#         profit = (close_price.iloc[i+1]-initial_price)/initial_price
#         if close_price.iloc[i+1]<last_low:
#             if profit1 != 0:
#                 return (profit1+profit)/2
#             else:
#                 return profit
#         if profit >=  0.17:
#             profit1 = 0.17
#         last_low = low_price.iloc[i]
#     if profit1 != 0:
#         return (profit1+profit)/2
#     else:
#         return profit
def strategy6_profit(stock, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    last_profit = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if all([last_profit<0,profit<0]):
            return profit
        if profit <-0.06:
            profit = -0.06
            return profit
        last_profit = profit
    return profit
def strategy7_profit(stock:pd.DataFrame, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if profit > 0.32:
                profit = 0.32
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.06:
                profit = -0.06
                break
            elif profit > 0.16:
                profit1 = 0.16
    return profit
def strategy8_profit(stock:pd.DataFrame, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if profit > 0.40:
                profit = 0.40
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.08:
                profit = -0.08
                break
            elif profit > 0.20:
                profit1 = 0.20
    return profit
d = -94
day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
day2M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+61))) , '%Y-%m-%d' )
day3M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+92))) , '%Y-%m-%d' )
print(f'day:{day}, day1M:{day1M}, day2M:{day2M}, day3M:{day3M}')
stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{"6585"}.TW.csv', parse_dates=['Date'], index_col=['Date'])
print('strategy9 1M 2M 3M')
print(strategy9_profit(stock, day, day1M))
print('--------------------------------')
print(strategy9_profit(stock, day, day2M))
print('--------------------------------')
print(strategy9_profit(stock, day, day3M))
print('--------------------------------')
print('strategyM11 1M 2M 3M')
print(strategyM11_profit(stock, day, day1M))
print('--------------------------------')
print(strategyM11_profit(stock, day, day2M))
print('--------------------------------')
print(strategyM11_profit(stock, day, day3M))
print('--------------------------------')
print('strategyM21 1M 2M 3M')
print(strategyM21_profit(stock, day, day1M))
print('--------------------------------')
print(strategyM21_profit(stock, day, day2M))
print('--------------------------------')
print(strategyM21_profit(stock, day, day3M))
print('--------------------------------')
print('strategyM31 1M 2M 3M')
print(strategyM31_profit(stock, day, day1M))
print('--------------------------------')
print(strategyM31_profit(stock, day, day2M))
print('--------------------------------')
print(strategyM31_profit(stock, day, day3M))

day:2023-08-14 00:00:00, day1M:2023-09-14 00:00:00, day2M:2023-10-14 00:00:00, day3M:2023-11-14 00:00:00
strategy9 1M 2M 3M
0.1479040070630374
--------------------------------
0.1479040070630374
--------------------------------
0.1479040070630374
--------------------------------
strategyM11 1M 2M 3M
79.699997
82.300003
condition12 0.04119074576461435
condition5MA 0.028432544528582807
condition10MA 0.02381527981232269
0.09343857010551984
--------------------------------
79.699997
82.300003
condition12 0.04119074576461435
condition5MA 0.028432544528582807
condition10MA 0.02381527981232269
0.09343857010551983
--------------------------------
79.699997
82.300003
condition12 0.04119074576461435
condition5MA 0.028432544528582807
condition10MA 0.02381527981232269
0.09343857010551983
--------------------------------
strategyM21 1M 2M 3M
condition10MA 0.017861459859242017
condition15 0.04993923390257955
0.20243007889076253
--------------------------------
condition10MA 0.017861459859242017
cond

## Variable Set

In [4]:
initial_day = -5008
final_day = -39
initial_date = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(initial_day))) , '%Y-%m-%d' )
final_date = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(final_day))) , '%Y-%m-%d' )
print(f'initial_date: {initial_date}')
print(f'final_date: {final_date}')

initial_date: 2010-02-28 00:00:00
final_date: 2023-10-07 00:00:00


In [6]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
print(taiwan0050.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ID', '5MA',
       '10MA', '20MA', '50MA', '100MA', '150MA', '200MA', '200MA ROCP',
       '200MA ROCP 20MA', '200MA ROCP 60MA', '5Max', '250Max', '5Min',
       '250Min', 'Volume 50MA', 'ROCP', 'OBV', 'beta', 'RS value', 'RS 250MA',
       'RS 50MA', 'RS 20MA', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA',
       'RS 250MA 10MAX', 'RS 50MA 10MAX', 'RS 20MA 10MAX', 'RS 250EMA 10MAX',
       'RS 50EMA 10MAX', 'RS 20EMA 10MAX', 'RS EMA250rate', 'RS EMA50rate',
       'RS EMA20rate', 'RS 250rate', 'RS 50rate', 'RS 20rate',
       'RS 250MA is 10MAX', 'RS 50MA is 10MAX', 'RS 20MA is 10MAX',
       'RS 250EMA is 10MAX', 'RS 50EMA is 10MAX', 'RS 20EMA is 10MAX',
       'RS 250MA 20MAX', 'RS 50MA 20MAX', 'RS 20MA 20MAX', 'RS 250MA is 20MAX',
       'RS 50MA is 20MAX', 'RS 20MA is 20MAX', 'RS 250EMA 20MAX',
       'RS 50EMA 20MAX', 'RS 20EMA 20MAX', 'RS 250EMA is 20MAX',
       'RS 50EMA is 20MAX', 'RS 20EMA is 20MAX', 'RS 250MA 50MAX',
   

In [8]:
all_stock_df = pd.DataFrame([], columns=taiwan0050.columns)
for i, id in enumerate(stock_num):
    try:
        stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{id}.csv',
                            index_col='Date', parse_dates=['Date'])
        stock = stock.loc[stock.index>initial_date]
        stock = stock.loc[stock.index<final_date]
        if len(stock) > 0:
            all_stock_df = pd.concat([all_stock_df, stock], ignore_index=False)
    except:
        print(f'{id} is not found')
        continue


In [10]:
all_stock_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/all_stock_df.txt', encoding='utf-8-sig', sep='   ')

TypeError: "delimiter" must be a 1-character string

In [21]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5008, -39, 7)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            stockdata = all_stock_df.loc[all_stock_df['ID'] == id].loc[day]
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            print(e)
            
            pass
    allstock.dropna(inplace=True)
    allstock['ID'] = allstock['ID'].astype(int).astype(str)
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            #TM
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            # allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            # allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            #T5
            allstock.loc[id, 'RS 20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 250rate>55'] = (allstock.loc[id, 'RS 250rate'] > 55)
            allstock.loc[id, 'RS 250rate<75'] = (allstock.loc[id, 'RS 250rate'] < 75)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #, 'year low sort', 'year high sort'
            #T6-
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'Volume 50MA>250k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            #T11
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate'] > 75)
            allstock.loc[id, 'Volume > Volume50MA'] = (allstock.loc[id, 'Volume 50MA'] < allstock.loc[id, 'Volume'])
            #,'RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k'
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            #All Template
            allstock.loc[id, 'T5'] = all(allstock.loc[id, ['RS 20rate>85', 'RS 250rate>55', 'RS 250rate<75', 'year low sort', 'year high sort', 'Volume 50MA>100k']])
            allstock.loc[id, 'T6'] = all(allstock.loc[id, ['RS EMA250rate>80', 'RS EMA20rate>80', 'RS EMA20rate<99', 'Volume 50MA>250k','Volume > Volume50MA']])
            allstock.loc[id, 'T11'] = all(allstock.loc[id, ['RS EMA250rate>75','RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k','Volume > Volume50MA']])
            allstock.loc[id, 'TM'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS EMA250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 篩出本日期符合Template的股票ID
    allstock.dropna(inplace=True)
    T5_ID = allstock.loc[allstock['T5']].index.values.astype(str)
    T6_ID = allstock.loc[allstock['T6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=True).iloc[-50:].index.values.astype(str)
    T11_ID = allstock.loc[allstock['T11']].index.values.astype(str)
    TM_ID = allstock.loc[allstock['TM']].index.values.astype(str)
    
    T5_stock_num = len(T5_ID)
    T6_stock_num = len(T6_ID)
    T11_stock_num = len(T11_ID)
    TM_stock_num = len(TM_ID)
    print('刪除前  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')
    all_template_ID = list(set(np.concatenate([T5_ID,T6_ID,T11_ID,TM_ID])))
    # print(all_template_ID)
    apexstock = allstock.loc[all_template_ID, ['產業別', 'T5', 'T6', 'T11', 'TM'
                                            , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX'
                                            , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
    # 防止用濾網篩選前50個的template混亂
    apexstock[['T5','T6','T11','TM']] = False
    apexstock.loc[T5_ID,'T5'] = True
    apexstock.loc[T6_ID,'T6'] = True
    apexstock.loc[T11_ID,'T11'] = True
    apexstock.loc[TM_ID,'TM'] = True
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    column_name = ['產業別', 'T5', 'T6', 'T11', 'TM'
                , 'S250rate', 'S50rate', 'S20rate', 'ES250rate', 'ES50rate', 'ES20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price','ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX'
                , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

    apexstock.columns = column_name
    apexstock = apexstock.dropna()
    print(f'apexstock shape:{apexstock.shape}')
    
    profit_list = []
    IDs = apexstock.index.values
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    day2M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+61))) , '%Y-%m-%d' )
    day3M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+92))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        if ID in all_template_ID:
            # print(ID)
            # try:
            #     stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
            # except:
            #     try:
            #         stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            #     except Exception as e:
            #         print(e)
            #         try:
            #             apexstock.drop(ID, inplace=True)
            #             print(f'drop stock : {ID}')
            #         except:
            #             pass
            #         continue
            got_profit_list = ([ID in T6_ID, ID in T11_ID])
            # print(got_profit_list)
            try:
                # initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
                # oneM_price = stock.loc[stock.index<day1M, 'Adj Close'].iloc[-1]
                stock = all_stock_df.loc[all_stock_df['ID'] == ID]
                profit1 = round(strategy4_profit(stock, day, day1M),3)
                profit2 = round(strategy7_profit(stock, day, day1M),3) 
                profit3 = round(strategy5_profit(stock, day, day1M),3) 
                profit4 = round(strategy8_profit(stock, day, day1M),3) 
                profit5 = round(strategy9_profit(stock, day, day1M),3) 
                IDprofit = [ID]
                for temp in got_profit_list:
                    for strategy in [profit1, profit2, profit3, profit4, profit5]:
                        IDprofit.append(strategy*temp)
                        
                profit_list.append(IDprofit)
                # profit_list.append([ID, got_profit_list[0]*profit1, got_profit_list[0]*profit2, got_profit_list[0]*profit3, got_profit_list[0]*profit4
                #                     , got_profit_list[1]*profit1, got_profit_list[1]*profit2, got_profit_list[1]*profit3, got_profit_list[1]*profit4
                #                     , got_profit_list[2]*profit1, got_profit_list[2]*profit2, got_profit_list[2]*profit3, got_profit_list[2]*profit4
                #                     , got_profit_list[3]*profit1, got_profit_list[3]*profit2, got_profit_list[3]*profit3, got_profit_list[3]*profit4])
            except Exception as e:
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                    print(f'drop stock : {ID}')
                except:
                    pass
                continue
        else:
            pass
    T5_stock_num = len(apexstock.loc[apexstock['T5']].index.values)
    T6_stock_num = len(apexstock.loc[apexstock['T6']].index.values)
    T11_stock_num = len(apexstock.loc[apexstock['T11']].index.values)
    TM_stock_num = len(apexstock.loc[apexstock['TM']].index.values)
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    profit_column = ['T5 S4 profit', 'T5 S7 profit', 'T5 S5 profit', 'T5 S8 profit', 'T5 S9 profit'
                                                   , 'T6 S4 profit', 'T6 S7 profit', 'T6 S5 profit', 'T6 S8 profit', 'T6 S9 profit'
                                                   , 'T11 S4 profit', 'T11 S7 profit', 'T11 S5 profit', 'T11 S8 profit', 'T11 S9 profit'
                                                   , 'TM S4 profit', 'TM S7 profit', 'TM S5 profit', 'TM S8 profit', 'TM S9 profit']
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'T5 S4 profit', 'T5 S7 profit', 'T5 S5 profit', 'T5 S8 profit', 'T5 S9 profit'
                                                   , 'T6 S4 profit', 'T6 S7 profit', 'T6 S5 profit', 'T6 S8 profit', 'T6 S9 profit'
                                                   , 'T11 S4 profit', 'T11 S7 profit', 'T11 S5 profit', 'T11 S8 profit', 'T11 S9 profit'
                                                   , 'TM S4 profit', 'TM S7 profit', 'TM S5 profit', 'TM S8 profit', 'TM S9 profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, profit_column] \
    = profit_df.loc[apexstock.index.values,profit_column]
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose for all template (S45789))(1Week)/' + str(day).split(' ')[0] + 'all template選股(S45789)(1Week)' + '.xlsx', encoding='utf-8-sig')
    profit = {'T5 S4 profit':100*profit_df.loc[apexstock.index.values,'T5 S4 profit'].sum()/T5_stock_num
              ,'T5 S7 profit':100*profit_df.loc[apexstock.index.values,'T5 S7 profit'].sum()/T5_stock_num
              ,'T5 S5 profit':100*profit_df.loc[apexstock.index.values,'T5 S5 profit'].sum()/T5_stock_num
              ,'T5 S8 profit':100*profit_df.loc[apexstock.index.values,'T5 S8 profit'].sum()/T5_stock_num
              ,'T5 S9 profit':100*profit_df.loc[apexstock.index.values,'T5 S9 profit'].sum()/T5_stock_num
              ,'T5 numbers':T5_stock_num
              ,'T6 S4 profit':100*profit_df.loc[apexstock.index.values,'T6 S4 profit'].sum()/T6_stock_num
              ,'T6 S7 profit':100*profit_df.loc[apexstock.index.values,'T6 S7 profit'].sum()/T6_stock_num
              ,'T6 S5 profit':100*profit_df.loc[apexstock.index.values,'T6 S5 profit'].sum()/T6_stock_num
              ,'T6 S8 profit':100*profit_df.loc[apexstock.index.values,'T6 S8 profit'].sum()/T6_stock_num
              ,'T6 S9 profit':100*profit_df.loc[apexstock.index.values,'T6 S9 profit'].sum()/T6_stock_num
              ,'T6 numbers':T6_stock_num
              ,'T11 S4 profit':100*profit_df.loc[apexstock.index.values,'T11 S4 profit'].sum()/T11_stock_num
              ,'T11 S7 profit':100*profit_df.loc[apexstock.index.values,'T11 S7 profit'].sum()/T11_stock_num
              ,'T11 S5 profit':100*profit_df.loc[apexstock.index.values,'T11 S5 profit'].sum()/T11_stock_num
              ,'T11 S8 profit':100*profit_df.loc[apexstock.index.values,'T11 S8 profit'].sum()/T11_stock_num
              ,'T11 S9 profit':100*profit_df.loc[apexstock.index.values,'T11 S9 profit'].sum()/T11_stock_num
              ,'T11 numbers':T11_stock_num
              ,'TM S4 profit':100*profit_df.loc[apexstock.index.values,'TM S4 profit'].sum()/TM_stock_num
              ,'TM S7 profit':100*profit_df.loc[apexstock.index.values,'TM S7 profit'].sum()/TM_stock_num
              ,'TM S5 profit':100*profit_df.loc[apexstock.index.values,'TM S5 profit'].sum()/TM_stock_num
              ,'TM S8 profit':100*profit_df.loc[apexstock.index.values,'TM S8 profit'].sum()/TM_stock_num
              ,'TM S9 profit':100*profit_df.loc[apexstock.index.values,'TM S9 profit'].sum()/TM_stock_num
              ,'TM numbers':TM_stock_num}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
        all_df.fillna(0,inplace=True)
    clear_output()
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    print('day', day)
    all_df_nonumbers = all_df.drop(['T5 numbers','T6 numbers','T11 numbers','TM numbers'], axis = 1)
    display(all_df_nonumbers)
    df_sum = all_df_nonumbers.sum()
    df_mean = all_df_nonumbers.mean()
    df_sum = pd.concat([df_sum,df_mean], axis=1)
    display(df_sum.transpose())

刪除後  T5 股票數量 :  42 | T6 股票數量 :  50 | T11 股票數量 :  59 | TM 股票數量 :  286 | apexstock shape : (327, 44)
day 2023-06-29 00:00:00


,T5 S4 profit,T5 S7 profit,T5 S5 profit,T5 S8 profit,T5 S9 profit,T6 S4 profit,T6 S7 profit,T6 S5 profit,T6 S8 profit,T6 S9 profit,T11 S4 profit,T11 S7 profit,T11 S5 profit,T11 S8 profit,T11 S9 profit,TM S4 profit,TM S7 profit,TM S5 profit,TM S8 profit,TM S9 profit
2023-06-29,-0.695238,-0.500000,0.071429,-0.123810,1.190476,2.276,3.306,3.616,2.506,6.684,1.237288,1.933898,2.386441,1.655932,2.684746,2.012587,2.386014,2.945105,2.550699,4.962238
2023-06-26,6.072222,6.455556,8.011111,7.627778,10.408333,5.884,6.060,6.706,6.450,11.816,4.024444,4.497778,5.340000,4.866667,7.122222,3.873498,4.364311,5.042403,4.532862,7.536396
2023-06-15,4.190476,3.695238,4.409524,4.904762,4.314286,3.126,3.160,3.924,3.890,5.304,4.225000,3.150000,4.103846,5.178846,5.430769,2.056897,2.059310,2.706897,2.704483,3.561724
2023-06-08,0.838298,0.940426,1.221277,1.119149,0.597872,2.894,3.208,3.794,3.480,4.098,-2.207018,-1.670175,-1.798246,-2.335088,-1.533333,1.456229,1.399327,1.574074,1.684848,2.103704
2023-06-01,6.616216,6.721622,7.867568,7.762162,10.391892,8.328,7.612,9.188,10.024,8.176,3.910638,3.848936,4.646809,4.708511,4.329787,3.750515,3.708935,4.137801,4.200000,4.024399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-12-23,-0.725000,0.275000,-1.250000,-2.250000,1.387500,1.328,1.480,1.974,1.942,2.480,2.745455,2.459091,3.004545,3.290909,4.750000,2.550000,2.310000,2.722000,3.082000,2.998000
2009-12-16,2.609091,3.700000,4.245455,3.154545,3.363636,4.096,3.746,4.106,4.456,3.780,2.989474,2.328947,2.802632,3.621053,1.663158,6.894828,6.837931,7.562069,7.618966,6.591379
2009-12-09,5.115385,5.576923,6.500000,6.038462,6.538462,1.944,2.162,2.572,2.354,6.984,5.235556,5.068889,4.731111,4.897778,9.624444,7.190741,6.800000,7.272222,7.662963,7.770370
2009-12-02,12.966667,12.966667,14.466667,14.466667,15.900000,11.160,10.396,11.794,12.558,9.536,12.592857,12.640476,13.988095,13.940476,18.761905,12.800000,12.921154,14.325000,14.203846,15.753846


,T5 S4 profit,T5 S7 profit,T5 S5 profit,T5 S8 profit,T5 S9 profit,T6 S4 profit,T6 S7 profit,T6 S5 profit,T6 S8 profit,T6 S9 profit,T11 S4 profit,T11 S7 profit,T11 S5 profit,T11 S8 profit,T11 S9 profit,TM S4 profit,TM S7 profit,TM S5 profit,TM S8 profit,TM S9 profit
0,741.444213,896.252601,1126.649122,966.438826,1618.362368,891.418330,993.792446,1150.809510,1056.205394,1436.343489,955.064582,1039.447145,1150.163782,1068.359760,1334.950616,1068.007419,1176.776289,1382.369774,1279.785439,1721.033648
1,1.125105,1.360019,1.709634,1.466523,2.455785,1.255519,1.399708,1.620858,1.487613,2.023019,1.345161,1.464010,1.619949,1.504732,1.880212,1.504236,1.657431,1.947000,1.802515,2.423991


In [22]:
all_df.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\all template(S45789)(1Week)1M profit.xlsx')